In [9]:
# each TF-Record file is for a single class and will be loaded into a separate tf.data.Dataset
# these datasets will be appended to a list, and fed into tf.experimental.sample_from_datasets

In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import gc

In [11]:
# directories for Train and Test TF Records, 264 files in each
train_dir = '/content/drive/My Drive/lala1/Data/Train'
test_dir = '/content/drive/My Drive/lala1/Data/Test'

In [12]:
train_tfr = os.listdir(train_dir)
test_tfr = os.listdir(test_dir)

In [13]:
def read_tfrecord(serialized_example):
    feature_description = {
          'feature0': tf.io.FixedLenFeature((), tf.string),
          'feature1': tf.io.FixedLenFeature((), tf.int64),
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)

    feature0 = tf.io.parse_tensor(example['feature0'], out_type = tf.float32)
    feature1 = example['feature1']

    return feature0, feature1

In [15]:
i = 0
lis = [0]*264
for d in train_tfr:
    d_path = os.path.join(train_dir, d)
    tfrecord_dataset_train = tf.data.TFRecordDataset([d_path], compression_type="GZIP")
    dataset = tfrecord_dataset_train.map(read_tfrecord)
    lis[i] = dataset
    i = i+1

train_ds = tf.data.experimental.sample_from_datasets(lis)

In [16]:
print(i)#264

264


In [17]:
i = 0
lis = [0]*264
for d in test_tfr:
    d_path = os.path.join(test_dir, d)
    tfrecord_dataset_test = tf.data.TFRecordDataset([d_path], compression_type="GZIP")
    dataset = tfrecord_dataset_test.map(read_tfrecord)
    lis[i] = dataset
    i = i+1

test_ds = tf.data.experimental.sample_from_datasets(lis)

In [18]:
print(i)#264

264


In [19]:
def prepare_for_training(ds, shuffle_buffer_size=64, batch_size=4):
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.batch(batch_size)#.repeat(2)
    ds = ds.map(setshape)
    ds = ds.prefetch(5)
    return ds

def setshape(x, y):
    x.set_shape([None, 900000,1])
    y = tf.expand_dims(y, -1)
    y.set_shape([None,1])
    return x, y

In [20]:
train_final = prepare_for_training(train_ds)
test_final = prepare_for_training(test_ds)

In [ ]:
# clearing RAM space by deleting the variables no longer needed, and calling the garbage collector

In [21]:
del train_dir
del test_dir
del d
del i
del train_ds
del test_ds
del test_tfr
del train_tfr
del d_path
del dataset
gc.collect()

0

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[900000, 1]))
for rate in (1, 2, 4) * 2:
    model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', dilation_rate=rate))
    model.add(tf.keras.layers.MaxPooling1D(4))
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=5))
model.add(tf.keras.layers.MaxPooling1D(4))
#model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(264, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 900000, 32)        192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 225000, 32)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 225000, 32)        5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 56250, 32)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 56250, 32)         5152      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 14062, 32)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 14062, 32)         5

In [23]:
m_cp = tf.keras.callbacks.ModelCheckpoint('Model-2', monitor='val_loss', verbose=0, save_best_only=True)

In [ ]:
history = model.fit_generator(train_final, epochs=30, validation_data=test_final, callbacks=[m_cp])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
   9098/Unknown - 1268s 139ms/step - loss: 4.2253 - sparse_categorical_accuracy: 0.0842